In [52]:
s = {}
for row in arr:
    for cell in row:
        c = f"{cell[0]},{cell[1]},{cell[2]}"
        if c not in s:
            s[c] = 1
        else:
            s[c] += 1

In [169]:
import numpy as np
from PIL import Image

class Color():
    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

    def get_diff(self, r, g, b):
        return abs(r-self.r) + abs(g-self.g) + abs(b-self.b)
    
class Elevation():
    def __init__(self, c, e):
        if type(c) != Color:
            raise TypeError("c should be of type Color")
        self.c = c
        self.e = e
        self.higher = None
        self.lower = None
    
    def set_higher(self, e):
        self.higher = e
    
    def set_lower(self, e):
        self.lower = e
        
    def get_diff(self, r, g, b):
        return self.c.get_diff(r, g, b)

class ElevationMap():
    MIN_LAT = 1.23776
    MIN_LON = 103.61751
    MAX_LAT = 1.47066
    MAX_LON = 104.04360
    D_LAT = MAX_LAT - MIN_LAT
    D_LON = MAX_LON - MIN_LON
    IMG_H = 1030
    IMG_W = 1885
        
    def __init__(self):
        self._set_elevations()
        img = Image.open("../Geospatial/singapore-elevation.png")
        self.arr = np.array(img.convert('RGB'))
        
    def _set_elevations(self):
        self.elevations = []
        
        c_map = {
            168: Color(255, 255, 255),#
            163: Color(219, 203, 201),#
            156: Color(216, 191, 186),#
            141: Color(217, 153, 151),#
            133: Color(224, 147, 145),#
            126: Color(214, 143, 135),#
            113: Color(218, 162, 132),#
            105: Color(185, 134, 118),#
            91: Color(179, 218, 146),#
            76: Color(202, 220, 139),#
            73: Color(180, 219, 130),#
            69: Color(178, 221, 130),#
            54: Color(167, 231, 139),#
            50: Color(164, 223, 150),#
            44: Color(149, 220, 142),#
            43: Color(147, 225, 134),#
            42: Color(145, 224, 152),#
            38: Color(148, 225, 171),#
            30: Color(146, 219, 197),#
            25: Color(140, 212, 221),#
            23: Color(167, 221, 230),#
            18: Color(136, 185, 220),#
            17: Color(148, 191, 223),#
            15: Color(162, 196, 228),#
            14: Color(153, 196, 241),#
            12: Color(135, 168, 218),#
            10: Color(157, 195, 249),#
            8: Color(137, 168, 216),#
            7: Color(138, 156, 219),#
            3: Color(130, 129, 219),#
            2: Color(143, 136, 222),#
            0: Color(134, 122, 239)#
        }
        
        curr_e = None
        for key, value in c_map.items():
            e = Elevation(value, key)
            e.set_higher(curr_e)
            curr_e = e
            self.elevations.append(e)
            
        curr_e = None
        for e in self.elevations[::-1]:
            e.set_lower(curr_e)
            curr_e = e
            
    def get_elevation_from_color(self, r, g, b):
        min_diff = float("inf")
        result_e = None
        for e in self.elevations:
            diff = e.get_diff(r, g, b)
            if min_diff > diff:
                min_diff = diff
                result_e = e
        lower = float("inf")
        higher = float("inf")
        if result_e.lower != None:
            lower = result_e.lower.get_diff(r, g, b)
        if result_e.higher != None:
            higher = result_e.higher.get_diff(r, g, b)
        if lower < higher:
            e = result_e.e
            e_2 = result_e.lower.e
            return (e*lower + e_2*min_diff)/(lower+min_diff)
        else:
            e = result_e.e
            e_2 = result_e.higher.e
            return (e*higher + e_2*min_diff)/(higher+min_diff)
        
    def get_elevation(self, lat, long):
        row = int((lat-ElevationMap.MIN_LAT)
                  *ElevationMap.IMG_H
                  /ElevationMap.D_LAT)
        col = int((long-ElevationMap.MIN_LON)
                  *ElevationMap.IMG_W
                  /ElevationMap.D_LON)
        return round(self.get_elevation_from_color(
            *self.arr[row][col]), 1)
    

In [167]:
e = ElevationMap()

In [168]:
e.get_elevation(1.3510570739548033, 103.88337018948374)

[167 231 247]


23.5